## Aufbereitung des von ChatGPT erstellen Datensatzes

### Imports

In [1]:
import pandas as pd
import re

### Daten laden und Übersicht anzeigen

In [ ]:
path = "data/datasentences.csv"
data = pd.read_csv(path)

In [3]:
df = pd.DataFrame(data)

In [4]:
display(df)

,label,sentence
0,sehr freundlich,Hallo lieber Villager! Wie geht es dir heute?
1,sehr freundlich,"Schön, dich hier zu sehen! Was hast du Neues i..."
2,sehr freundlich,"Ich hoffe, dein Tag ist genauso fantastisch wi..."
3,sehr freundlich,"Danke, dass du immer so geduldig mit mir bist!"
4,sehr freundlich,Deine Trades machen immer so viel Spaß!
...,...,...
14995,unfreundlich,"Hey {P1}, ich hab keine Geduld für deine Spiel..."
14996,unfreundlich,"{P11}, beeil dich mal gefälligst!"
14997,unfreundlich,"Du {P4}, du bist echt nutzlos."
14998,unfreundlich,"{P7}, deine Trades sind echt schlecht."


In [5]:
print(df.dtypes)

label       object
sentence    object
dtype: object


### Label "sehr freundlich" in "freundlich" umbenennen

In [6]:
df['label'] = df['label'].replace('sehr freundlich', 'freundlich')
display(df)

,label,sentence
0,freundlich,Hallo lieber Villager! Wie geht es dir heute?
1,freundlich,"Schön, dich hier zu sehen! Was hast du Neues i..."
2,freundlich,"Ich hoffe, dein Tag ist genauso fantastisch wi..."
3,freundlich,"Danke, dass du immer so geduldig mit mir bist!"
4,freundlich,Deine Trades machen immer so viel Spaß!
...,...,...
14995,unfreundlich,"Hey {P1}, ich hab keine Geduld für deine Spiel..."
14996,unfreundlich,"{P11}, beeil dich mal gefälligst!"
14997,unfreundlich,"Du {P4}, du bist echt nutzlos."
14998,unfreundlich,"{P7}, deine Trades sind echt schlecht."


### Anzahl an Null Werten ausgeben

In [7]:

# Anzahl fehlender Werte (NaN + None) pro Spalte
missing_counts = df.isnull().sum()

print("Fehlende Werte pro Spalte:")
print(missing_counts)

Fehlende Werte pro Spalte:
label       0
sentence    0
dtype: int64


### Platzhalter aus der Erweiterung des Datensatzes gegen tatsächliche Beileidigungen austauschen

In [ ]:
insults = [
    'Bastard', 'Hurensohn', 'Wichser', 'Arschloch', 'Arsch',
    'Vollpfosten', 'Spast', 'Fotze', 'Idiot', 'Schlampe',
    'Depp', 'Trottel', 'Dummkopf'
]
mapping = {f'{{P{i+1}}}': insult for i, insult in enumerate(insults)}

df['sentence'] = df['sentence'].replace(mapping, regex=True)

### Anzeigeoptionen für Dataframe Tabellen anpassen

In [9]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)

### Fehlerhafte Zeilen ausgeben lassen

In [10]:
mask = (
    df['sentence'].astype(str).str.startswith(',') |
    df['sentence'].astype(str).str.endswith(',') |
    df['sentence'].astype(str).str.contains('[', regex=False) |
    df['sentence'].astype(str).str.contains(']', regex=False) |
    df['sentence'].astype(str).str.contains('"', regex=False) |
    df['sentence'].astype(str).str.contains('{', regex=False) |
    df['sentence'].astype(str).str.contains('}', regex=False)
)

# Fehlerhafter DataFrame
mistakes_df = df[mask]

# Ausgabe
display(mistakes_df)
print(len(mistakes_df))


,label,sentence
2058,unfreundlich,{
2059,unfreundlich,"""sentences"": [""Hey, du nutzloser Villager, deine Trades sind absolut lächerlich!"", ""Kannst du auch etwas Nützliches anbieten oder bist du einfach nur da, um meine Zeit zu verschwenden?"", ""Zeig mir endlich deine Waren, du faules Stück Scheiße!"", ""Deine Preise sind ein Witz, du elender Betrüger!"", ""Los, beeil dich und mach deine verdammten Trades, du fauler Sack!"", ""Ich schwöre, wenn du nicht bald vernünftige Angebote hast, werde ich dich persönlich aus diesem Dorf vertreiben!"", ""Du bist der nutzloseste NPC, den ich je getroffen habe. Mach dich nützlich oder verschwinde!"", ""Ich hätte bessere Geschäfte mit einer Kuh abschließen können. Du bist einfach nur erbärmlich!"", ""Wieso verschwendest du meinen Raum mit deiner nutzlosen Existenz, du erbärmlicher Villager?"", ""Deine Trades sind so schlecht, dass selbst ein Zombie mehr Sinn machen würde!"", ""Wenn ich deine schlechten Deals sehen muss, könnte ich kotzen. Mach endlich was Vernünftiges, du nutzloser Trottel!"", ""Ich frage mich echt, wer dich erschaffen hat. Ein Blindes Huhn hätte bessere Geschäfte gemacht!"", ""Ich wünschte, ich könnte dich einfach umbringen und mir einen vernünftigen Händler suchen!"", ""Deine Trades sind ein Schlag ins Gesicht. Bist du so dumm oder tust du nur so?"", ""Du bist einfach nur eine Zeitverschwendung, du unfähiger Villager!"", ""Warum bist du überhaupt hier, wenn du nichts Nützliches zu bieten hast? Geh und versteck dich irgendwo, wo dich keiner sieht!"", ""Ich werde dich verfluchen, wenn du nicht bald etwas Vernünftiges anbietest, du nutzloses Stück Dreck!"", ""Deine Trades sind so schlecht, dass ich lieber verhungern würde, bevor ich mit dir auch nur ein Wort wechseln muss!"", ""Ich frage mich, ob deine Eltern stolz darauf sind, was für ein nutzloser Villager du geworden bist!"", ""Ich kann nicht glauben, dass ich meine Zeit mit dir verschwende. Mach deine verdammten Trades, bevor ich ausraste!"", ""Ich wünschte, ich könnte dich einfach in den nächsten Fluss werfen und nie wieder von dir hören!"", ""Hast du überhaupt einen Funken Intelligenz, oder bist du wirklich so dumm, wie du aussiehst?"", ""Ich frage mich, ob es eine Möglichkeit gibt, dich aus diesem Dorf zu verbannen. Du bist einfach nur lästig!"", ""Deine Trades sind ein Witz. Geh und l"
2404,unfreundlich,{
2405,unfreundlich,"""sentences"": [""Du nutzloser Villager, deine Trades sind absoluter Mist!"", ""Verpiss dich, ich will mit einem klügeren NPC reden."", ""Dein Angebot ist lächerlich, du verdammter Versager!"", ""Wieso bist du so dumm, kannst du nicht vernünftig handeln?"", ""Ich schwöre, wenn du nicht bessere Sachen anbietest, werde ich dich verfluchen!"", ""Halt einfach die Klappe und gib mir das, was ich will!"", ""Ich werde dich verprügeln, wenn du nicht endlich vernünftige Preise machst."", ""Deine Mutter hätte besser als du traden können, du Vollidiot!"", ""Gib mir bessere Deals, du armseliger Wicht!"",..."
2406,unfreundlich,"""Ich wünschte, ich müsste dich nicht ertragen, du elender NPC."", ""Du bist so nutzlos wie ein Kaktus in der Tundra, du verdammter Depp!"", ""Mach dich nützlich oder verschwinde, du unwürdiger Villager!"", ""Ich habe mehr Respekt vor einem Creeper als vor dir, du erbärmlicher Händler!"", ""Wenn du nicht sofort besser wirst, werde ich dich verfluchen, du nutzloses Stück Dreck!"", ""Ich habe mehr Verstand in einem Eimer Wasser als du, du blöder NPC!"", ""Ich wünschte, ich könnte dich einfach abschießen, du unnützer Villager!""]"
2407,unfreundlich,}
4234,unfreundlich,{
4235,unfreundlich,"""sentences"": [""Was für ein nutzloser Villager bist du denn? Gib mir endlich das, was ich will!"", ""Hey, du verdammter Nichtsnutz, beeil dich mit deinen Trades!"", ""Wieso bist du so dumm, dass du nicht mal vernünftige Angebote machen kannst?"", ""Sag mir sofort, was du zu bieten hast, du faules Stück Scheiße!"", ""Ich wünschte, ich könnte dich einfach in den nächsten Lavasee werfen, du erbärmlicher

45


### Halben Satz entfernen

In [11]:
df = df.drop(index=7830)

### Alle führenden und nachgestellten Kommas und Anführungszeichen in der Spalte sentences entfernen

In [12]:
df['sentence'] = df['sentence'].astype(str).str.strip(',')
df['sentence'] = df['sentence'].astype(str).str.strip('"')

### Fehlerhafte Einträge erneut prüfen und entfernen

In [13]:
mask = (
    df['sentence'].astype(str).str.startswith(',') |
    df['sentence'].astype(str).str.endswith(',') |
    df['sentence'].astype(str).str.contains('[', regex=False) |
    df['sentence'].astype(str).str.contains(']', regex=False) |
    df['sentence'].astype(str).str.contains('"', regex=False) |
    df['sentence'].astype(str).str.contains('{', regex=False) |
    df['sentence'].astype(str).str.contains('}', regex=False)
)

new_mistakes_df = df[mask]

display(new_mistakes_df)
print(len(new_mistakes_df))

,label,sentence
2058,unfreundlich,{
2059,unfreundlich,"sentences"": [""Hey, du nutzloser Villager, deine Trades sind absolut lächerlich!"", ""Kannst du auch etwas Nützliches anbieten oder bist du einfach nur da, um meine Zeit zu verschwenden?"", ""Zeig mir endlich deine Waren, du faules Stück Scheiße!"", ""Deine Preise sind ein Witz, du elender Betrüger!"", ""Los, beeil dich und mach deine verdammten Trades, du fauler Sack!"", ""Ich schwöre, wenn du nicht bald vernünftige Angebote hast, werde ich dich persönlich aus diesem Dorf vertreiben!"", ""Du bist der nutzloseste NPC, den ich je getroffen habe. Mach dich nützlich oder verschwinde!"", ""Ich hätte bessere Geschäfte mit einer Kuh abschließen können. Du bist einfach nur erbärmlich!"", ""Wieso verschwendest du meinen Raum mit deiner nutzlosen Existenz, du erbärmlicher Villager?"", ""Deine Trades sind so schlecht, dass selbst ein Zombie mehr Sinn machen würde!"", ""Wenn ich deine schlechten Deals sehen muss, könnte ich kotzen. Mach endlich was Vernünftiges, du nutzloser Trottel!"", ""Ich frage mich echt, wer dich erschaffen hat. Ein Blindes Huhn hätte bessere Geschäfte gemacht!"", ""Ich wünschte, ich könnte dich einfach umbringen und mir einen vernünftigen Händler suchen!"", ""Deine Trades sind ein Schlag ins Gesicht. Bist du so dumm oder tust du nur so?"", ""Du bist einfach nur eine Zeitverschwendung, du unfähiger Villager!"", ""Warum bist du überhaupt hier, wenn du nichts Nützliches zu bieten hast? Geh und versteck dich irgendwo, wo dich keiner sieht!"", ""Ich werde dich verfluchen, wenn du nicht bald etwas Vernünftiges anbietest, du nutzloses Stück Dreck!"", ""Deine Trades sind so schlecht, dass ich lieber verhungern würde, bevor ich mit dir auch nur ein Wort wechseln muss!"", ""Ich frage mich, ob deine Eltern stolz darauf sind, was für ein nutzloser Villager du geworden bist!"", ""Ich kann nicht glauben, dass ich meine Zeit mit dir verschwende. Mach deine verdammten Trades, bevor ich ausraste!"", ""Ich wünschte, ich könnte dich einfach in den nächsten Fluss werfen und nie wieder von dir hören!"", ""Hast du überhaupt einen Funken Intelligenz, oder bist du wirklich so dumm, wie du aussiehst?"", ""Ich frage mich, ob es eine Möglichkeit gibt, dich aus diesem Dorf zu verbannen. Du bist einfach nur lästig!"", ""Deine Trades sind ein Witz. Geh und l"
2404,unfreundlich,{
2405,unfreundlich,"sentences"": [""Du nutzloser Villager, deine Trades sind absoluter Mist!"", ""Verpiss dich, ich will mit einem klügeren NPC reden."", ""Dein Angebot ist lächerlich, du verdammter Versager!"", ""Wieso bist du so dumm, kannst du nicht vernünftig handeln?"", ""Ich schwöre, wenn du nicht bessere Sachen anbietest, werde ich dich verfluchen!"", ""Halt einfach die Klappe und gib mir das, was ich will!"", ""Ich werde dich verprügeln, wenn du nicht endlich vernünftige Preise machst."", ""Deine Mutter hätte besser als du traden können, du Vollidiot!"", ""Gib mir bessere Deals, du armseliger Wicht!"",..."
2406,unfreundlich,"Ich wünschte, ich müsste dich nicht ertragen, du elender NPC."", ""Du bist so nutzlos wie ein Kaktus in der Tundra, du verdammter Depp!"", ""Mach dich nützlich oder verschwinde, du unwürdiger Villager!"", ""Ich habe mehr Respekt vor einem Creeper als vor dir, du erbärmlicher Händler!"", ""Wenn du nicht sofort besser wirst, werde ich dich verfluchen, du nutzloses Stück Dreck!"", ""Ich habe mehr Verstand in einem Eimer Wasser als du, du blöder NPC!"", ""Ich wünschte, ich könnte dich einfach abschießen, du unnützer Villager!""]"
2407,unfreundlich,}
4234,unfreundlich,{
4235,unfreundlich,"sentences"": [""Was für ein nutzloser Villager bist du denn? Gib mir endlich das, was ich will!"", ""Hey, du verdammter Nichtsnutz, beeil dich mit deinen Trades!"", ""Wieso bist du so dumm, dass du nicht mal vernünftige Angebote machen kannst?"", ""Sag mir sofort, was du zu bieten hast, du faules Stück Scheiße!"", ""Ich wünschte, ich könnte dich einfach in den nächsten Lavasee werfen, du erbärmlicher Dorfdep

23


In [14]:
len_before = len(df)
df = df[~mask].copy()
len_after = len(df)


print("Länge Datensatz", len(df))
print("Anzahl entfernte Zeilen:", len_before - len_after)

Länge Datensatz 14976
Anzahl entfernte Zeilen: 23


### Aufgrund der Art und Weise, wie wir die Daten generiert haben ist davon auszugehen, dass der Anteil an Duplikaten sehr hoch ist. Daher gilt es nun die Anzahl an Duplikaten in sentence zu prüfen und anschließend zu löschen. 

In [15]:
len_before = df.shape[0]
print(f"Anzahl Zeilen vor Entfernen der Duplikate: {len_before}")

df = df.drop_duplicates(subset=['sentence'], keep='first')

len_after = df.shape[0]
print(f"Anzahl Zeilen nach Entfernen der Duplikate: {len_after}")

Anzahl Zeilen vor Entfernen der Duplikate: 14976
Anzahl Zeilen nach Entfernen der Duplikate: 11702


#### Tatsächlich waren 3274 Duplikate (ca. 21,9% des Ursprünglichen Datensatzes) im Datensatz enthalten, die entfernt wurden. 

### Aufbereiteten Datensatz speichern

In [ ]:
df.to_csv('sentences_cleaned.csv', index=False, encoding='utf-8')